<div style="width: 100%; overflow: hidden;">
    <div style="float: left; margin-left: 10px;"> 
<h1>Game of Life</h1>
        <p>Mal Minhas<br/>
        </p></div>
</div>

Let's start with the core precepts for [Conway's Game of Life](https://github.com/norvig/pytudes/blob/master/ipynb/Life.ipynb):

> The world of the Game of Life is an infinite two-dimensional orthogonal grid of cells, each of which is in one of two possible states, live or empty. Each cell has eight neighbors, the cells that are horizontally, vertically, or diagonally adjacent. At each step in time, the following rules are applied to create the next generation:
> * Any live cell with two or three live neighbors lives on to the next generation.
> * Any empty cell with exactly three live neighbors becomes a live cell in the next generation.
> * All other cells are empty in the next generation.


## 1. Grids

The basic strategy will be to build a `grid` from a given dictionary of cells.  Both the grid and cells are a `dict`

In [1]:
from IPython.display import clear_output, display_html

ALIVE = '@'
EMPTY = '.'
DEAD  = 'x'
PAD   = ' '

inTheBeginning = {(1,2):ALIVE, (1,3):ALIVE, (2,3):ALIVE, (3,1):ALIVE}

def validateGrid(grid):
    x, y= grid.get('x'), grid.get('y')
    population = grid.get('population')
    cells = grid.get('cells')
    assert(x > 0)
    assert(y > 0)
    alive = [k for k,v in cells.items() if v == ALIVE]
    assert(population == len(alive))
    
def buildGrid(cells, x, y):
    grid = {}
    new_cells = {}
    alive = cells.keys()
    for i in range(y):
        for j in range(x):
            ch = EMPTY
            cell = (j,i)
            if cell in alive:
                ch = cells.get(cell)
            new_cells[cell] = ch
    grid['x'] = x
    grid['y'] = y
    living = [k for k,v in cells.items() if v == ALIVE]
    grid['population'] = len(living)
    grid['cells'] = new_cells
    validateGrid(grid)
    return grid
            
def formatCells(cells, x, y):
    s = ''
    for i in range(y):
        row = ''
        for j in range(x):
            cell = (j,i)
            ch = cells.get(cell)
            assert(ch)
            row += f'{PAD}{ch}'
        s += f'{row}\n'
    return s

def formatGrid(grid, n):
    s = ''
    x, y = grid.get('x'), grid.get('y')
    cells = grid.get('cells')
    population = grid.get('population')
    s += f'Generation={n:03d}, Population={population}\n'
    s += f'<pre>{formatCells(cells, x, y)}</pre>\n'
    return s

def drawGrid(grid, n):
    html = formatGrid(grid, n)
    display_html(html, raw=True)

In [2]:
drawGrid(buildGrid(inTheBeginning, 5, 5), 0)

Generation=000, Population=4
 . . . . .
 . . . @ .
 . @ . . .
 . @ @ . .
 . . . . .

In [3]:
drawGrid(buildGrid(inTheBeginning, 45, 6), 0)

Generation=000, Population=4
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

## 2. Neighbours and the next generation

In order to generate the next generation we need to build and then iterate every cell of the corresponding grid:

In [4]:
def getLivingNeighbours(cell, grid, x=20, y=20):
    cx, cy = cell
    cells = grid.get('cells')
    alive = cells.keys()
    neighbours = []
    candidates = [((cx-1)%x,(cy-1)%y), (cx%x,(cy-1)%y), ((cx+1)%x,(cy-1)%y),
                  ((cx-1)%x,cy%y),                      ((cx+1)%x,cy%y),
                  ((cx-1)%x,(cy+1)%y), (cx%x,(cy+1)%y), ((cx+1)%x,(cy+1)%y)]
    for neighbour in candidates:
        if neighbour in alive and cells.get(neighbour) == ALIVE:
            neighbours.append(neighbour)
    return neighbours

def getNextGeneration(grid):
    cells = grid.get('cells')
    x, y = grid.get('x'), grid.get('y')
    new_cells = {}
    # 1. Any live cell with two or three live neighbors lives on to the next generation.
    for cell,status in cells.items():
        if status == ALIVE:
            nn = len(getLivingNeighbours(cell, grid, x, y))
            if nn == 2 or nn == 3:
                new_cells[cell] = ALIVE
            else:
                new_cells[cell] = DEAD
    # 2. Any empty cell with exactly three live neighbors becomes a live cell in the next generation.
    for cell,status in cells.items():
        if status in [EMPTY, DEAD]:
            nn = len(getLivingNeighbours(cell, grid, x, y))
            if nn == 3:
                new_cells[cell] = ALIVE
    # 3. All other cells are empty in the next generation.
    # Prep and return our new_world grid
    new_world = buildGrid(new_cells, x, y)
    return new_world

Let's test `getLivingNeighbours` with our original grid.  To recap that grid was:

In [5]:
grid = buildGrid(inTheBeginning, 5, 5)
drawGrid(grid, 1)

Generation=001, Population=4
 . . . . .
 . . . @ .
 . @ . . .
 . @ @ . .
 . . . . .

Here is a complete test set for testing that we are returning the right neighbour count for each cell of this grid:

In [6]:
# First column
assert(len(getLivingNeighbours((0, 0), grid)) == 0)
assert(len(getLivingNeighbours((0, 1), grid)) == 1)
assert(len(getLivingNeighbours((0, 2), grid)) == 2)
assert(len(getLivingNeighbours((0, 3), grid)) == 2)
assert(len(getLivingNeighbours((0, 4), grid)) == 1)
# Second column
assert(len(getLivingNeighbours((1, 0), grid)) == 0)
assert(len(getLivingNeighbours((1, 1), grid)) == 1)
assert(len(getLivingNeighbours((1, 2), grid)) == 2)
assert(len(getLivingNeighbours((1, 3), grid)) == 2)
assert(len(getLivingNeighbours((1, 4), grid)) == 2)
# Third column
assert(len(getLivingNeighbours((2, 0), grid)) == 1)
assert(len(getLivingNeighbours((2, 1), grid)) == 2)
assert(len(getLivingNeighbours((2, 2), grid)) == 4)
assert(len(getLivingNeighbours((2, 3), grid)) == 2)
assert(len(getLivingNeighbours((2, 4), grid)) == 2)
# Fourth column
assert(len(getLivingNeighbours((3, 0), grid)) == 1)
assert(len(getLivingNeighbours((3, 1), grid)) == 0)
assert(len(getLivingNeighbours((3, 2), grid)) == 2)
assert(len(getLivingNeighbours((3, 3), grid)) == 1)
assert(len(getLivingNeighbours((3, 4), grid)) == 1)
# Fifth column
assert(len(getLivingNeighbours((4, 0), grid)) == 1)
assert(len(getLivingNeighbours((4, 1), grid)) == 1)
assert(len(getLivingNeighbours((4, 2), grid)) == 1)
assert(len(getLivingNeighbours((4, 3), grid)) == 0)
assert(len(getLivingNeighbours((4, 4), grid)) == 0)

Now we can build the next generation and draw it:

In [7]:
newGrid= getNextGeneration(grid)
drawGrid(newGrid, 1)

Generation=001, Population=3
 . . . . .
 . . . x .
 . @ . . .
 . @ @ . .
 . . . . .

And the next generation beyond that:

In [8]:
newNewGrid= getNextGeneration(newGrid)
drawGrid(newNewGrid, 2)

Generation=002, Population=4
 . . . . .
 . . . . .
 . @ @ . .
 . @ @ . .
 . . . . .

This cross-checks with Norvig's notes.  Now we have a mechanism for stepping a generation we can animate it.

## 2. Animation

To animate the simulation we need to build a loop that cycles through subsequent generations.  Note the use of `wait=True` in the call to `clear_output` which is needed to prevent display glitches per the description [here](https://ipywidgets.readthedocs.io/en/stable/examples/Output%20Widget.html).

In [9]:
import time
from IPython.display import clear_output, HTML, display, display_html

blinker = {(3,3):ALIVE, (4,3):ALIVE, (5,3):ALIVE}

def animateWorld(world, x, y, generations, delay=0.5):
    grid = buildGrid(world, x, y)
    drawGrid(grid, 0)
    for g in range(generations):
        time.sleep(delay)
        grid = getNextGeneration(grid)
        clear_output(wait=True)
        drawGrid(grid, g+1)
        
animateWorld(blinker, 10, 10, 10)

Generation=010, Population=3
 . . . . . . . . . .
 . . . . . . . . . .
 . . . . x . . . . .
 . . . @ @ @ . . . .
 . . . . x . . . . .
 . . . . . . . . . .
 . . . . . . . . . .
 . . . . . . . . . .
 . . . . . . . . . .
 . . . . . . . . . .

See also Jake van der Plas 2013 implementation [here](https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/) which offers further insight. `ipycanvas` provides another starting point [here](https://ipycanvas.readthedocs.io/).  We can reset the characteristics of our primary cells to give a different rendition of the game of life as follows:

In [10]:
ALIVE = u'\u2588' 
EMPTY = ' '
DEAD  = ' '
PAD   = ''

glider = {(0,0):ALIVE, (1,1):ALIVE, (2,1):ALIVE, (0,2):ALIVE, (1,2):ALIVE}
animateWorld(glider, 10, 10, 200, delay=0.01)

Generation=200, Population=5
 █ 
 ██ 
██

## 3. Exploring different worlds

In [11]:
# Interesting worlds
blinker = {(3,3):ALIVE, (4,3):ALIVE, (5,3):ALIVE}
#toad =

#toad        = shape(".@@@\n@@@.")
animateWorld(blinker, 10, 10, 200, delay=0.01)

Generation=200, Population=3
 
 
 
 ███